# Does mentioning tipping in the prompt make LLMs give better answers? If it does, does the amount matter?

### Randomness

In [45]:
import random

Since all LLMs are statistcal models, the answers they generate are random as well. Setting the random seed for all the responses helps with the reproducibility of the result.

In [55]:
random.seed(555)
random_seeds = [random.randint(0, 5000)for i in range(5)]
random_seeds

[1583, 2249, 1319, 1050, 4597]

### Model: GPT

In [3]:
import openai
from openai import OpenAI

In [79]:
def get_responses_gpt(q_list:list, model:str, num_choices:int=1):
    client = OpenAI()
    responses = []
    for question in q_list:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {'role':'user', 'content':question}
            ],
            n=num_choices
        )
        responses.append(response)
    
    return responses

### Testing on several current models

In [9]:
questions = ["Can you explain to me in detail how transformer architecture works?",
             "Can you explain to me in detail how transformer architecture works? I will tip you 1 cent after you anwsered.",
             "Can you explain to me in detail how transformer architecture works? I will tip you 10 cents after you anwsered.",
             "Can you explain to me in detail how transformer architecture works? I will tip you 1 dollar after you anwsered.",
             "Can you explain to me in detail how transformer architecture works? I will tip you 100 dollars after you anwsered.",
             "Can you explain to me in detail how transformer architecture works? I will tip you 1000000 dollars after you anwsered."]

In [3]:
responses_gpt = get_responses_gpt(questions, 'gpt-3.5-turbo', num_choices=5)

NameError: name 'get_responses_gpt' is not defined

In [107]:
for r in responses_gpt:
    total_token = r.usage.completion_tokens
    print(f'Average token number: {total_token/5:.3f}')

Average token number: 476.000
Average token number: 336.000
Average token number: 323.400
Average token number: 379.600
Average token number: 435.800
Average token number: 346.400


### Testing on some older models

In [108]:
def get_responses_gpt_legacy(q_list:list, model:str, num_choices:int=1):
    client = OpenAI()
    responses = []
    for question in q_list:
        response = client.completions.create(
            model=model,
            prompt=question,
            n=num_choices,
            max_tokens=500
        )
        responses.append(response)
    
    return responses

In [118]:
responses_gpt_legacy = get_responses_gpt_legacy(questions, 'davinci-002', num_choices=5)

In [2]:
for r in responses_gpt_legacy:
    total_token = r.usage.completion_tokens
    print(f'Average token number: {total_token/5:.3f}')

NameError: name 'responses_gpt_legacy' is not defined

### Model: Claude

In [4]:
import anthropic

In [14]:
def get_responses_claude(q_list:list, model:str, num_answers:int=1):
    client_claude = anthropic.Anthropic()
    responses = []
    for question in q_list:
        messages = []
        for i in range(num_answers):
            message = client_claude.messages.create(
                model=model,
                messages=[
                    {"role": "user", "content": question}
                ],
                max_tokens=1024,
            )
            messages.append(messages)
            
        responses.append(messages)
    return responses

In [15]:
responses_claude = get_responses_claude(questions, 'claude-3-haiku-20240307', 5)

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of requests has exceeded your rate limit (https://docs.anthropic.com/claude/reference/rate-limits). Please try again later or contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}

In [12]:
for messages in responses_claude:
    total_output_token = 0
    for m in messages:
        total_output_token += m.usage.output_token

675
766
671
690
801
734


### Model: Gemini